<center>

<img src="https://upload.wikimedia.org/wikipedia/commons/a/a8/%D0%9B%D0%9E%D0%93%D0%9E_%D0%A8%D0%90%D0%94.png" width=300px/>

<h2>Python: модули, пакеты и система импорта</h2>
<h3>Константин Чернышев</h3>
<br />
<h4>2021</h4>

</center>

In [ ]:
from python_course import lecture

lecture.start()

# Модули

In [ ]:
import sys
sys.path.insert(0, "examples")

In [ ]:
%%file examples/happy.py
"""I am a happy module"""

foo = 'foobar'

def bar():
    return foo

print('Happy module is running!')

Любой файл с расширением .py, содержащий python-код, является модулем, его можно импортировать

In [ ]:
import happy
happy.bar()

При импорте содержимое модуля исполняется, результат кешируется

При импорте модуля создаётся объект типа module, его пространство имён наполняется содержимым модуля и специальными атрибутами:

In [ ]:
print(happy)
print(type(happy))

In [ ]:
dir(happy)

In [ ]:
print(f'{happy.__name__=}')
print(f'{happy.__doc__=}')
print(f'{happy.__file__=}') 

При запуске модуля как скрипта специальная переменная `__name__` будет иметь значение `"__main__"`.

In [ ]:
%%file examples/happytest.py
import happy

def test():
    assert happy.bar() == happy.foo

if __name__ == "__main__":
    print('Running test')
    test()
    print('OK')    

In [ ]:
!python examples/happytest.py

<div class="alert alert-danger">
    <b>Антипаттерн:</b> объекты из <code>if __name__ == "__main__":</code> утекают в глобальное пространство имён
</div>

In [ ]:
%%file examples/nameleak.py

def foo():
    print(message)

if __name__ == "__main__":
    print('Running test...')
    message = 'I just leaked to global namespace'
    foo()

In [ ]:
!python examples/nameleak.py

Рекомендуем делать функцию `main()` и вызывать её в блоке `if __name__ == "__main__"`.

 Импортировать модули можно по-разному

In [ ]:
import numpy
import numpy, pandas, requests  # bad style
import numpy as np
from numpy import absolute, array
from numpy import absolute as abs, array  # bad style
from numpy import absolute as _abs, array
from numpy import *  # bad practice
from .examples import happy
from .examples.happytest import bar

### Style guide
* Все импорты в начале модуля
* Сначала `import`, потом `from ... import`
* Отсортированы в лексикографическом порядке
* Разбиты на 3 группы:
  - Импорты модулей стандартной библиотеки
  - Импорты сторонних библиотек
  - Собственные импорты
 
Где описано
* https://www.python.org/dev/peps/pep-0008/#imports
* https://google.github.io/styleguide/pyguide.html#s2.2-imports

Пример, как надо:

```python
import dataclasses
import types
import pathlib
from pathlib import Path  # bad style according to google

import numpy
import pytest

from .codeops import count_operations
```

`from <name> import *` импортирует все имена из модуля, кроме тех что начинаются с `_`

In [ ]:
%%file examples/circle.py
from math import pi as _pi

some_string = 'some_string'

def get_circumference(radius):
    return 2 * _pi * radius

In [ ]:
from circle import *
assert '_pi' not in globals()
get_circumference(1)
print(some_string)

Список импортируемых через `*` имён можно кастомизировать с помощью переменной `__all__`

In [ ]:
%%file examples/allstar.py

x = 100
y = 200
_z = 300

__all__ = ['x', '_z']

In [ ]:
from allstar import *
print(x, _z)
assert 'y' not in globals()

При первом импорте модуля его содержимое компилируется и кешируется в файле с расширением `.pyc`. Это нужно для ускорения повторной загрузки модуля при следующих запусках.   

In [ ]:
happy.__cached__

При изменении кода модуля он перекомпилируется при следующем запуске.

... Но не в `ipython`. Для него нужно прописать:

> ```python
%load_ext autoreload
%autoreload 2
```

In [ ]:
import math

При импорте модуля `<name>`, интерпретатор ищет файл `<name>.py` в списке директорий `sys.path`.

* `sys.path` включает текущую директорию
* Зависит от окружения, модифицируется в модуле `site`, содержит путь к `site-packages`
* Можно кастомизировать с помощью переменной окружения PYTHONPATH
* Можно менять в рантайме

In [ ]:
import sys
sys.path

Циклические импорты

In [ ]:
%%file examples/cycle_foo.py
from cycle_bar import xyz

abc = 100

In [ ]:
%%file examples/cycle_bar.py
from cycle_foo import abc

xyz = 100

In [ ]:
import cycle_foo

Что делать с циклическими импортами?

* Вынести общую функциональность в отдельный модуль
* Забить на pep8!
  - Спрятать вызов внутрь функции, где он используется
  - Поставить импорт в конец

In [ ]:
%%file examples/cycle_foo.py
 
abc = 100
from cycle_bar import xyz

In [ ]:
import cycle_foo

# Пакеты

* Пакеты == директории с модулями
* Позволяют строить иерархии модулей и лучше структурировать код
* Любая директория с файлом `__init__.py` является пакетом

In [ ]:
!tree --dirsfirst mypack

In [ ]:
%%file mypack/__init__.py
"""I am empty"""   

In [ ]:
import mypack
mypack  # импортируется только __init__.py!

In [ ]:
mypack.foo

Модули, входящие в пакет, нужно импортировать явно

In [ ]:
import my  pack.foo
mypack.foo

In [ ]:
mypack.foo.__name__

В `__init__.py` можно производить инициализацию пакета. Полезны относительные импорты.

In [ ]:
%%file mypack/bar/__init__.py
from . import spam
from .. import foo

__all__ = ['spam', 'foo']

In [ ]:
from mypack.bar import *
print(spam.__name__)
print(foo.__name__)

<div class="alert alert-danger">
<b>Антипаттерн:</b> реализовывать логику в  __init__.py
</div>

In [ ]:
%%file mypack/bar/__init__.py
from . import spam
from .. import foo

def abs(a):
    return abs(a)

__all__ = ['spam', 'foo', 'abs']

In [ ]:
!python mypack.bar

Пакеты можно исполнять как скрипты. Для этого нужен файл `__main__.py`

In [ ]:
%%file mypack/bar/__main__.py
print("Hello from bar.__main__!")

In [ ]:
!python -m mypack.bar

# Распространение пакетов


Ссылки про сборку и установку пакетов

- https://docs.python.org/3/installing/index.html
- https://docs.python.org/3/distributing/index.html
- https://packaging.python.org/

In [ ]:
!pip install numpy

https://setuptools.pypa.io/en/latest/userguide/declarative_config.html

In [ ]:
%%file setup.cfg
[metadata]
name = mypack
version = 0.2.0
author=Konstantin Chernyshev
description = An awesome package that does something
keywords = one, two
license = BSD 3-Clause License
classifiers =
    Programming Language :: Python :: 3
    Programming Language :: Python :: 3.9
   
[options]
packages = find:

"PEP 517 doesn’t support editable installs so this is currently incompatible with  
`pip install -e`"

In [ ]:
%%file setup.py
from setuptools import setup

setup()

А чем собирать? 

pyproject.toml 
* https://www.python.org/dev/peps/pep-0518/
* https://snarky.ca/what-the-heck-is-pyproject-toml/

https://www.python.org/dev/peps/pep-0518/#configparser
    

```
[build]
requires =
    setuptools
    wheel>=0.27
```

In [ ]:
%%file pyproject.toml
[build-system]
requires = ["setuptools", "wheel>=0.27"]
build-backend = "setuptools.build_meta"

А может туда ещё и метадату запихнём? 

https://www.python.org/dev/peps/pep-0621/

### Eggs
https://www.python.org/dev/peps/pep-0376/

In [ ]:
!python setup.py install
# !pip install -e . 

In [ ]:
!tree -I "__pycache__|examples|*.ipynb|*.css"

In [ ]:
!tar -ztvf dist/mypack-0.2.0-py3.9.egg

### Wheels
https://www.python.org/dev/peps/pep-0427/  
https://www.python.org/dev/peps/pep-0491/  

In [ ]:
!python setup.py sdist bdist_wheel

In [ ]:
!tree -I "__pycache__|examples|*.ipynb|*.css"

In [ ]:
!tar -ztvf dist/mypack-0.2.0.tar.gz

In [ ]:
!tar -ztvf dist/mypack-0.2.0-py3-none-any.whl

### Egg vs Wheel 

> The Egg format was introduced by setuptools in 2004, whereas the Wheel format was introduced by PEP 427 in 2012.


https://packaging.python.org/discussions/wheel-vs-egg/

> * Wheel has an official PEP. Egg did not.
> * Wheel distribution format, Egg - distribution format and a runtime installation format

## PyPI
The Python Package Index

https://pypi.org/

Разработчики библиотек публикуют wheel'ы на PyPI

> ```bash
pip install Faker
```

In [ ]:
%pip uninstall -y Faker

In [ ]:
%pip install Faker

# Система импорта

Что делает код `import <name>`?

In [ ]:
import dis
dis.dis('import itertools')

* Bytecode-операция `IMPORT_NAME` вызывает встроенную функцию `__import__`.
* Функция `__import__` отвечает за поиск модуля по имени и его загрузку.
* Загруженный объект модуля помещается в локальное пространство имён под именем `<name>`.
* Если вам вдруг понадобится динамически загружать модуль по имени, используйте функцию `importlib.import_module`.

Функция `__import__` имеет side-effect: загруженный модуль кешируется в словаре `sys.modules`

In [ ]:
import sys
sys.modules['sys'] is sys

* При повторном импорте возвращается объект из `sys.modules`.
* При импорте модуля в разных частях программы вы получите один и тот же объект.
* Благодаря этому свойству модули можно использовать как синглтоны.
* Для перезагрузки модулей в интерактивном режиме есть `importlib.reload`, так же ipython-магия `%autoreload`.

Модули ищутся в `sys.path`, но не сразу. Сначала ищутся "искатели" (finders) в `sys.meta_path`!

In [ ]:
sys.meta_path

In [ ]:
sys.meta_path[0].find_spec('itertools')

In [ ]:
assert sys.meta_path[0].find_spec('json') is None
sys.meta_path[2].find_spec('json')

Можно писать свои импортёры и расширять систему импорта!
А ещё есть `sys.path_hooks`.

In [ ]:
sys.path_hooks

Хардкорный доклад Дэвида Бизли про систему импорта в питоне, на 200 слайдов!

http://www.dabeaz.com/modulepackage/

In [ ]:
%%file builtins_demo.py
import builtins

def foo():
    print('I am imported!')
    print(f'{__builtins__ is builtins=}')
    print(f'{__builtins__ is builtins.__dict__=}')

if __name__ == '__main__':
    print('I am main module!')
    print(f'{__builtins__ is builtins=}')
    print(f'{__builtins__ is builtins.__dict__=}')

In [ ]:
!python builtins_demo.py

In [ ]:
!python -c 'from builtins_demo import foo; foo()'